Import Module

In [1]:
from glob import glob
from keras_unet_collection import models, utils
from PIL import Image
from shapely.affinity import affine_transform
from shapely.geometry import MultiPolygon, Polygon
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from utils import *

import cv2
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio
import shutil
import sys
import tifffile as tiff
import warnings

2023-10-01 21:01:12.180867: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-01 21:01:12.214089: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7630] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 21:01:12.214122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-01 21:01:12.214151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-01 21:01:12.221400: I tensorflow/core/platform/cpu_feature_g

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


2023-10-01 21:01:12.866844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Prevent Visualization Error

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# 재귀 깊이 제한
sys.setrecursionlimit(10**6)

2023-10-01 20:52:41.743913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1833] Created device /device:GPU:0 with 2 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:17:00.0, compute capability: 8.0


In [3]:
# ourModel : model_name, rate, input_channels, delete_channel, hyperparams를 입력으로 받음
# 고정 : train/test/valid set
# 변수 : model_name, rate, input_channels, delete_channel, hyperparams
class ourModel():
    def __init__(self, case_name, model_name, input_data_path):
        # Constructor to initialize the model
        # Loads input data from a JSON file
        self.case_name = case_name
        self.model_name = model_name
        self.base_path = '/home/u2018144071/SIG/result'
        
        # Loads input data from a JSON file
        with open(input_data_path, 'r') as file:
            data = json.load(file)

        self.input_names = np.array(sorted(data["input_names"]))
        self.label_names = np.array(sorted(data["label_names"]))
        self.train_input_names = np.array(sorted(data["train_input_names"]))
        self.train_label_names = np.array(sorted(data["train_label_names"]))
        self.valid_input_names = np.array(sorted(data["valid_input_names"]))
        self.valid_label_names = np.array(sorted(data["valid_label_names"]))
        self.test_input_names = np.array(sorted(data["test_input_names"]))
        self.test_label_names = np.array(sorted(data["test_label_names"]))

        print("Number of images with supraglacial lake : {0}".format(len(self.input_names)))
        print("Training:validation:testing = {}:{}:{}".format(len(self.train_label_names), len(self.valid_label_names), len(self.test_label_names)))   

    def data_pre_processing(self, size, img_channels, input_channels):
        # Pre-processes the data for training
        self.delete_channel = [x for x in [i for i in range(img_channels)] if x not in input_channels]
        self.input_channels = len(input_channels)
        self.img_channels = img_channels
        self.size = size

        self.valid_input = input_data_process(image_to_array(self.valid_input_names, self.size, self.img_channels), self.delete_channel)
        self.valid_label = target_data_process(image_to_array(self.valid_label_names, self.size, channel=1))
    
        self.test_input = input_data_process(image_to_array(self.test_input_names, self.size, self.img_channels), self.delete_channel)
        self.test_label = target_data_process(image_to_array(self.test_label_names, self.size, channel=1))
    
    # 해당 case에 대한 model_training
    # hyperparameter tuning 가능!

    def model_training(self, params_path):
        # Trains the model based on the provided hyperparameters
        with open(params_path, 'r') as file:
            params = json.load(file)
        
        # params로부터 model의 hyperparameter를 받음
        filter_num_ = params["filter_num"]
        activation_ = params["activation"]
        atten_activation_ = params["atten_activation"]
        attention_ = params["attention"]
        output_activation_ = params["output_activation"]
        batch_norm_ = params["batch_norm"]
        pool_ = params["pool"]
        unpool_ = params["unpool"]
        backbone_ = params["backbone"]
        weights_ = params["weights"]
        freeze_backbone_ = params["freeze_backbone"]
        freeze_batch_norm_ = params["freeze_batch_norm"]
        optimizer_ = params["optimizer"]
        learning_rate_ = params["learning_rate"]
        N_epoch = params["N_epoch"]
        N_batch = params["N_batch"]
        N_sample = params["N_sample"]
        max_tol = params["max_tol"]
        min_del = params["min_del"]

        self.params = params
        
        model = models.att_unet_2d((256, 256, self.input_channels), filter_num=filter_num_, n_labels=2, 
                                stack_num_down=2, stack_num_up=2, activation=activation_, 
                                atten_activation=atten_activation_, attention=attention_, output_activation=output_activation_, 
                                batch_norm=batch_norm_, pool=pool_, unpool=unpool_, 
                                backbone=backbone_, weights=weights_, 
                                freeze_backbone=freeze_backbone_, freeze_batch_norm=freeze_batch_norm_, 
                                name='attunet')
        
        if optimizer_ == "SGD":
            optimizer_ = keras.optimizers.SGD(learning_rate=learning_rate_)
            
        elif optimizer_ == "Adam":
            optimizer_ = keras.optimizers.Adam(learning_rate=learning_rate_)
            
        model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer_)
        
        L_train = int(0.6*len(self.input_names))

        tol = 0
        # loop over epoches
        for epoch in range(N_epoch):
            # initial loss record
            if epoch == 0:
                y_pred = model.predict([self.valid_input])
                record = np.mean(keras.losses.categorical_crossentropy(self.valid_label, y_pred))
                print('\tInitial loss = {}'.format(record))
            
            # loop over batches
            for step in range(N_batch): 
                # selecting smaples for the current batch
                ind_train_shuffle = utils.shuffle_ind(L_train)[:N_sample] 
                
                # batch data formation
                ## augmentation is not applied
                train_input = input_data_process(image_to_array(self.train_input_names[ind_train_shuffle], self.size, self.img_channels), self.delete_channel)
                train_label = target_data_process(image_to_array(self.train_label_names[ind_train_shuffle], self.size, channel=1))
                
                # train on batch
                loss_ = model.train_on_batch([train_input,], [train_label,])
                # ** training loss is not stored ** #
                
            # epoch-end validation
            y_pred = model.predict([self.valid_input])
            record_temp = np.mean(keras.losses.categorical_crossentropy(self.valid_label, y_pred))
            # ** validation loss is not stored ** #
            
            # if loss is reduced
            if record - record_temp > min_del:
                print('Validation performance is improved from {} to {}'.format(record, record_temp))
                record = record_temp; # update the loss record
                tol = 0; # refresh early stopping patience 
                # ** model checkpoint is not stored ** #
                
            # if loss not reduced
            else:
                print('Validation performance {} is NOT improved'.format(record_temp))
                tol += 1
                if tol >= max_tol:
                    print('Early stopping with only {0} epochs.'.format(epoch))
                    break;
                else:
                    # Pass to the next epoch
                    continue;
                    
        print("학습 종료")
            
        self.model = model
    
    def save_model(self):
        # Saves the trained model
        # hyperparameter, model name, threshold, f1_score will save to json file.
        model_save_path = os.path.join(self.base_path, self.case_name, self.model_name)
        
        os.makedirs(model_save_path, exist_ok=True)
        self.model.save(os.path.join(model_save_path, self.model_name) + ".h5")

        print("모델 저장 완료. save path : {0}".format(model_save_path))

    def load_model(self, load_path):
        self.model = keras.models.load_model(load_path)

    def save_result(self):
        # Calculates evaluation metrics and saves results
        self.y_pred = self.model.predict([self.test_input])
    
        cross_entropy = np.mean(keras.losses.categorical_crossentropy(self.test_label, self.y_pred))
        threshold = find_best_threshold(self.test_label, self.y_pred)
        f1, best_img, worst_img = f1_score(self.test_label, self.y_pred, threshold)
        
        # save results
        self.result_dict = {}
        self.result_dict["cross_entropy"] = float(cross_entropy)
        self.result_dict["f1_score"] = float(f1)
        self.result_dict["threshold"] = float(threshold)
        self.result_dict["best_img"] = list(map(int, best_img))
        self.result_dict["worst_img"] = list(map(int, worst_img))

    def save_only_test_set(self):
        # Saves results for the test set
        threshold = self.result_dict["threshold"]
        y_result = np.where(self.y_pred < threshold, 1, 0)
        save_result(self.case_name, self.model_name, y_result, self.test_input_names, train=True, whole=True)

    
    def save_whole_train_region(self):
        # Saves results for the entire training region
        print("Train 전 지역에 대해 결과 저장 및 gpkg 생성 시작")
        threshold = self.result_dict["threshold"]
        reigion_input = input_data_process(image_to_array(self.input_names, self.size, self.img_channels), self.delete_channel)

        y_pred = self.model.predict([reigion_input])
        y_result = np.where(y_pred < threshold, 1, 0)
        save_result(self.case_name, self.model_name, y_result, self.input_names, train=True, whole=True)

    def save_whole_test_region(self, dict):
        # Saves results for specific test regions
        print("입력한 Test 지역에 대해 결과 저장 및 gpkg 생성 시작")
        threshold = self.result_dict["threshold"]
        test_region_input_names = make_nameslist(dict, image=True)
        print(len(test_region_input_names))

        flag_edge = split_samples_edge(test_region_input_names)
        test_region_input_names = test_region_input_names[flag_edge]
        print(len(test_region_input_names))

        flag_rock = split_samples_rock(test_region_input_names)
        test_region_input_names = test_region_input_names[flag_rock]
        print(len(test_region_input_names))

        flag_cloud = split_samples_cloud(test_region_input_names)
        test_region_input_names = test_region_input_names[flag_cloud]
        print(len(test_region_input_names))

        print(f"Let's make polygons with {len(test_region_input_names)} images from test regions")

        reigion_input = input_data_process(image_to_array(test_region_input_names, self.size, self.img_channels), self.delete_channel)

        y_pred = self.model.predict([reigion_input])
        y_result = np.where(y_pred < threshold, 1, 0)
        save_result(self.case_name, self.model_name, y_result, test_region_input_names, train=False, whole=True)

    def save_result_json(self):
        # Saves the evaluation metrics and hyperparameters as a JSON file
        json_save_path = os.path.join(self.base_path, self.case_name, self.model_name, self.model_name) + ".json"
        
        # Combine result_dict and params into a single dictionary
        data = {
            "result": self.result_dict,
            "params": self.params
        }

        with open(json_save_path, "w") as output_file:
            json.dump(data, output_file)

In [ ]:
params_path = "/home/u2018144071/SIG/params2.json"

img_channels = 8
size = 256
n_case = 2

channel_dict = {
    #"G" : [1],
    #"R" : [2],
    #"NIR" : [3],
    #"SWIR" : [4],
    #"NDWI" : [5],
    #"NDWI_ice" : [6],
    #"NDSI" : [7],
    #"NIR + NDWI_ice" : [3,6],
    #"NIR + NDWI + NDWI_ice" : [3,5,6],
    #"NIR + NDWI" : [3,5],
    #"NIR + NDWI + NDWI_ice + R": [2,3,5,6],
    #"NIR + NDWI + NDWI_ice + SWIR" : [3,4,5,6],
    "NIR + NDWI + NDWI_ice + R + SWIR" : [2,3,4,5,6],
    #"NIR + NDWI_ice + R" : [2,3,6,],
    #"NIR + NDWI_ice + R" : [2,3,6],
    #"NIR + NDWI + NDWI_ice + R" : [2,3,5,6],
    #"NIR + SWIR + NDWI + NDWI_ice + +NDSI + R" : [2,3,4,5,6,7]
}

total = 1
rate = 0.03

# dict : key : 날짜, items : region list 
#train_dict = {"0603" : [2,4,6], "0619" : [1,3,5], "0731" : [2,4,6], "0825" : [1,3,5]}    ###########
#train_dict = {"0603" : [2,4,6]} # train에 사용할 data를 dict 형태로 넣어주세요.
test_dict = {"0603" : [1,3,5]}  # 최종적으로 polygon을 생성할 data를 dict 형태로 넣어주세요.

for case in range(n_case):   # n_case개의 case를 진행
    
    case_name = "last_model_Case {0}_".format(case+1)
    #case_name = "0603_Case {0}_param_add".format(case+1)

    #input_data_path = data_subsetting(train_dict, case_name, rate)          
    input_data_path = "/home/u2018144071/SIG/result/decid_model_Case 2_/decid_model_Case 2_.json"   #####
    print(input_data_path)

    for model_name in channel_dict:
        print("{0}번째 학습/case_name : {1}/model name : {2}".format(total, case_name, model_name))
        test = ourModel(case_name, model_name, input_data_path)

        input_channels = channel_dict[model_name]
        test.data_pre_processing(size, img_channels, input_channels)

        test.model_training(params_path)
        test.save_model()
        test.save_result()
        test.save_result_json()
        
        if test.result_dict["f1_score"] > 0:
            test.save_only_test_set()
            #test.save_whole_train_region()
            test.save_whole_test_region(test_dict)   #######                 

        total += 1
        print("Finished!")

In [2]:
load_path = "/home/u2018144071/SIG/result/jj_model_Case 2_/NIR + NDWI + NDWI_ice + R + SWIR/NIR + NDWI + NDWI_ice + R + SWIR.h5"

input_data_path = "/home/u2018144071/SIG/result/jj_model_Case 2_/jj_model_Case 2_.json"
model_name = "NIR + NDWI + NDWI_ice + R + SWIR"
case_name = "Load"
input_channels = [2,3,4,5,6]
test_dict = {"0603" : [1,3,5]}


img_channels = 8
size = 256

test_region_input_names = make_nameslist(test_dict, image=True)
print(len(test_region_input_names))

flag_edge = split_samples_edge(test_region_input_names)
test_region_input_names = test_region_input_names[flag_edge]
print(len(test_region_input_names))

flag_rock = split_samples_rock(test_region_input_names)
test_region_input_names = test_region_input_names[flag_rock]
print(len(test_region_input_names))

flag_cloud = split_samples_cloud(test_region_input_names, rate=0.005)
test_region_input_names = test_region_input_names[flag_cloud]
print(len(test_region_input_names))

14684
11762
10213
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
94
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
77
0
0
0
0
0
0


Post Processing & 위에 Model학습은 가장 좋은 결과를 가지는 trainset을 보기위해 돌려둠

In [3]:
256*256*0.005

327.68

In [ ]:
result_path  = "merged_gpkg"
gdf = select_polygons_based_on_condition(result_path, threshold= 100000.0, buf_dist= 30)

Input Region Shape File path & read

In [ ]:
region1_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_1.shp"
region3_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_3.shp"
region5_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_5.shp"

region2_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_2.shp"
region4_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_4.shp"
region6_file = "C:/Users/scsi/Desktop/Jiwan/SIG/shapefile/region_num_6.shp"

region1 = gpd.read_file(region1_file)
region3 = gpd.read_file(region3_file)
region5 = gpd.read_file(region5_file)

region2 = gpd.read_file(region2_file)
region4 = gpd.read_file(region4_file)
region6 = gpd.read_file(region6_file)

In [ ]:
def add_region(gdf):
    
    intersection1 = gpd.overlay(region1, gdf, how = 'intersection' )
    intersection3 = gpd.overlay(region3, gdf, how = 'intersection' )
    intersection5 = gpd.overlay(region5, gdf, how = 'intersection' )

    intersection2 = gpd.overlay(region2, gdf, how = 'intersection' )
    intersection4 = gpd.overlay(region4, gdf, how = 'intersection' )
    intersection6 = gpd.overlay(region6, gdf, how = 'intersection' )

    intersection1 = intersection1.drop(columns=['fid'])
    intersection3 = intersection3.drop(columns=['fid'])
    intersection5 = intersection5.drop(columns=['fid'])

    intersection2 = intersection2.drop(columns=['fid'])
    intersection4 = intersection4.drop(columns=['fid'])
    intersection6 = intersection6.drop(columns=['fid'])

    merged_intersection = gpd.GeoDataFrame(pd.concat([intersection1, intersection3, intersection5,intersection2, intersection4, intersection6], ignore_index=True))

    return merged_intersection

#merged_intersection.to_file(save_path, driver="GPKG")

In [ ]:
gdf = add_region(gdf)

Input NDWI_ice image

In [ ]:
from shapely.geometry import box
from rasterio.features import geometry_mask

In [ ]:
NDWI_ice_path = ""

...mean value가 전부다 0.01정도로 잡혀.. 내가가진 tiff는 원본 파일로 해야할 것 같아..


In [ ]:
with rasterio.open(NDWI_path,'r+') as src:
    # MultiPolygon을 래스터 데이터의 좌표 참조 시스템에 맞게 재투영
    src.crs = gdf.crs

    # 빈 GeoDataFrame 초기화
    bbox_geometries = []

    # MultiPolygon의 경계 상자 추출
    for index, row in gdf.iterrows():
        multipolygon = row['geometry']
        # MultiPolygon의 경계 상자 추출
        bbox = box(*multipolygon.bounds)
        bbox_geometries.append(bbox)

    bbox_gdf = gpd.GeoDataFrame(geometry=bbox_geometries, crs=gdf.crs)

    # 경계 상자에 해당하는 GeoTIFF 값을 클리핑하고 평균 계산
    means = []
    for index, row in bbox_gdf.iterrows():
        bbox = row['geometry']
        mask = geometry_mask([bbox], out_shape=src.shape, transform=src.transform)
        values = src.read(1, masked=True)
        masked_values = values[mask]
        mean_value = masked_values.mean()
        print(mean_value)
        means.append(mean_value)

    bbox_gdf["mean_value"] = means

# # 평균 값이 특정 임계값 (예: 0.5) 이하인 Polygon을 삭제
threshold = 0.8
to_remove = bbox_gdf[bbox_gdf["mean_value"] <= threshold].index
gdf = gdf.drop(to_remove)

# GeoPackage 파일로 저장 (클리닝된 결과를 다시 저장)
output_gpkg_path = "C:/Users/scsi/Desktop/Jiwan/SIGCup/data/DeePLearning/add_attribute/step2.gpkg"
gdf.to_file(output_gpkg_path, driver="GPKG")